Question 5
Create a Model for Sentiment Analysis using RNN (Applying Embedding
Technique)

In [25]:
# Import required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [26]:
# Step 1: Load and Preprocess the dataset
max_features = 10000  # Number of words to consider as features
maxlen = 500  # Cut texts after this number of words (among the top max_features)

In [27]:
# Load the IMDb dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [28]:
# Pad sequences (to ensure equal length for all input sequences)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [29]:
# Step 2: Define the RNN model
model = Sequential()
model.add(Embedding(max_features, 32, input_length=maxlen))  # Embedding layer
model.add(SimpleRNN(32))  # RNN layer with 32 units
model.add(Dense(1, activation='sigmoid'))  # Output layer (for binary classification)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [30]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
# Step 3: Train the model
batch_size = 32
epochs = 10

In [8]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 73s 114ms/step - accuracy: 0.5788 - loss: 0.6654 - val_accuracy: 0.6952 - val_loss: 0.5817
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 81s 112ms/step - accuracy: 0.7737 - loss: 0.4930 - val_accuracy: 0.8004 - val_loss: 0.4628
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 94s 132ms/step - accuracy: 0.8733 - loss: 0.3107 - val_accuracy: 0.8012 - val_loss: 0.4606
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 133ms/step - accuracy: 0.9428 - loss: 0.1653 - val_accuracy: 0.8084 - val_loss: 0.5272
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 131s 115ms/step - accuracy: 0.9791 - loss: 0.0694 - val_accuracy: 0.7608 - val_loss: 0.7312
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 72s 116ms/step - accuracy: 0.9863 - loss: 0.0500 - val_accuracy: 0.6366 - val_loss: 0.6491
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 74s 118ms/step - accuracy: 0.7400 - loss: 0.5101 - val_accuracy: 0.7036 - val_loss: 0.5856
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 81s 116ms/step - accuracy: 0.8508 - loss:

In [9]:
# Step 4: Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.7183 - loss: 0.6487
Test Accuracy: 0.72


In [10]:
# Step 5: Save the model (optional)
model.save('sentiment_analysis_rnn_model.h5')
print("Model saved successfully.")

Model saved successfully.


**Question 3**
This dataset is a collection of images from the internet, played really amazing
player (So, can hope for perfection in shots). Then the data was augmented to give
rise to the data I provide in front of you.
https://www.kaggle.com/datasets/aneesh10/cricket-shot-dataset
General Information:
The directory drives consist of the cover drive, straight drive and off drive.
The directory legglance-flick contains the images for the leg glance and flick shot.
The directory pullshot has the images for pull shot.
The directory sweep has the image for sweep shot.
This dataset can be used for various purposes. It can be used for:
Apply any pre-trained model for classification of shot.
Note: This dataset is already augmented and some images may be pretty bad. I
would suggest to not augment it further

In [32]:
!pip install kaggle

In [33]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [34]:
!kaggle datasets download -d aneesh10/cricket-shot-dataset

Dataset URL: https://www.kaggle.com/datasets/aneesh10/cricket-shot-dataset
License(s): unknown
cricket-shot-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [35]:
# Unzip the dataset
with zipfile.ZipFile("cricket-shot-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("cricket_shot_dataset")

In [36]:
# Set up paths based on your dataset structure
data_dir = '/content/cricket_shot_dataset/data'

In [37]:
# Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Use 80% of data for training, 20% for validation
)

In [38]:
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)


Found 3779 images belonging to 4 classes.


In [39]:
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 944 images belonging to 4 classes.


In [40]:
# Load the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [41]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [42]:
# Add custom layers for fine-tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [43]:
# Build the model
model = Model(inputs=base_model.input, outputs=predictions)

In [44]:
# Compile the model with the correct argument
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [48]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=2
)


Epoch 1/2
118/118 ━━━━━━━━━━━━━━━━━━━━ 813s 7s/step - accuracy: 0.4079 - loss: 1.2918 - val_accuracy: 0.3901 - val_loss: 1.3231
Epoch 2/2
  1/118 ━━━━━━━━━━━━━━━━━━━━ 11:33 6s/step - accuracy: 0.3750 - loss: 1.2888

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


118/118 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.3750 - loss: 1.2888 - val_accuracy: 0.4375 - val_loss: 1.3203


In [49]:
# Evaluate the model on the validation set
validation_generator.reset()  # Reset the validation generator
preds = model.predict(validation_generator, steps=validation_generator.samples // validation_generator.batch_size + 1)


30/30 ━━━━━━━━━━━━━━━━━━━━ 173s 6s/step


In [50]:
# Get true labels
y_true = validation_generator.classes


In [51]:
# Convert predicted probabilities to class labels
y_pred = preds.argmax(axis=1)

In [52]:
# Generate classification report
print(classification_report(y_true, y_pred, target_names=validation_generator.class_indices.keys()))

                 precision    recall  f1-score   support

          drive       0.26      0.38      0.31       244
legglance-flick       0.27      0.12      0.16       224
       pullshot       0.26      0.24      0.25       252
          sweep       0.24      0.28      0.26       224

       accuracy                           0.26       944
      macro avg       0.26      0.25      0.25       944
   weighted avg       0.26      0.26      0.25       944



Question 4 B
.Draw a given model using keras Functional API    

In [53]:
from keras.layers import Input, Dense, Concatenate
from keras.models import Model

# Input layers
input_1 = Input(shape=(32,))
input_2 = Input(shape=(128,))

# Branch 1
dense_1 = Dense(8, activation='relu')(input_1)
dense_2 = Dense(4, activation='relu')(dense_1)

# Branch 2
dense_3 = Dense(64, activation='relu')(input_2)
dense_4 = Dense(32, activation='relu')(dense_3)
dense_5 = Dense(4, activation='relu')(dense_4)

# Concatenate
concatenated = Concatenate()([dense_2, dense_5])

# Final layers
dense_6 = Dense(2, activation='relu')(concatenated)
dense_7 = Dense(1, activation='sigmoid')(dense_6)

# Create the model
model = Model(inputs=[input_1, input_2], outputs=dense_7)